## Data Analysis

In [1]:
import os
import dotenv

dotenv.load_dotenv()
DB_HOST = os.getenv('GP_DB_HOST')
DB_PORT = os.getenv('GP_DB_PORT')
DB_USER = os.getenv('GP_DB_USER')
DB_PASS = os.getenv('GP_DB_PASS')
DB_NAME = os.getenv('GP_DB_NAME')

DB_HOST = "groupseven.c7uho3baxiu8.us-west-2.rds.amazonaws.com"
DB_NAME = "groupseven"
DB_USER = "seven"
DB_PASS = "BAIT580SQL"
DB_PORT = "5432"

conString = {'host':DB_HOST,
             'user':DB_USER,
             'dbname':DB_NAME,
             'password':DB_PASS,
             'port':DB_PORT}

# Data manipulation
import pandas as pd
import missingno as msno

import numpy as np
import psycopg2

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import altair as alt
import matplotlib as plt
alt.data_transformers.disable_max_rows()  # disable error for big datasets

DataTransformerRegistry.enable('default')

In [31]:
d = pd.read_excel('data/TIMMS2019/1-1_achievement-results-M4.xlsx',header=4)
x = d[['Country','Average \nScale Score']].iloc[0:60,:]
x.columns = ['Country','AVGScore']
x['year']='2019'
x.tail(20)

,Country,AVGScore,year
40,Albania,494.0,2019
41,New Zealand,487.0,2019
42,France,485.0,2019
43,Georgia,482.0,2019
44,United Arab Emirates,481.0,2019
45,Bahrain,480.0,2019
46,North Macedonia,472.0,2019
47,Montenegro,453.0,2019
48,Bosnia and Herzegovina,452.0,2019
49,Qatar,449.0,2019


In [35]:
d = pd.read_excel('data/worldbank_development_indicators.xlsx',sheet_name='Country - Metadata')
d

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,National accounts base year,National accounts reference year,System of National Accounts,SNA price valuation,Alternative conversion factor,PPP survey years,Balance of Payments Manual in use,External debt Reporting status,System of trade,Government Accounting concept,IMF data dissemination standard,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,2-alpha code,WB-2 code,Table Name,Short Name
0,AFG,Islamic State of Afghanistan,Low income,South Asia,IDA,HIPC,Afghan afghani,1979,"Demographic and Health Survey, 2015",Fiscal year end: March 20; reporting period fo...,2016,NaN,Country uses the 1993 System of National Accou...,Value added at basic prices (VAB),NaN,PPPs are imputed for 2011 and 2017 based on a ...,BPM6,Actual,General trade system,Consolidated central government,Enhanced General Data Dissemination System (e-...,"Integrated household survey (IHS), 2016/17",NaN,NaN,NaN,2018.0,AF,AF,Afghanistan,Afghanistan
1,AFE,Africa Eastern and Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"26 countries, stretching from the Red Sea in t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZH,ZH,Africa Eastern and Southern,Africa Eastern and Southern
2,AFW,Africa Western and Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"22 countries, stretching from the westernmost ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZI,ZI,Africa Western and Central,Africa Western and Central
3,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,Original chained constant price data are resca...,2010.0,Country uses the 2008 System of National Accou...,Value added at basic prices (VAB),NaN,Rolling surveys (annual estimation),BPM6,Actual,Special trade system,Consolidated central government,Enhanced General Data Dissemination System (e-...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,AL,AL,Albania,Albania
4,DZA,People's Democratic Republic of Algeria,Lower middle income,Middle East & North Africa,IBRD,NaN,Algerian dinar,2020 (expected),"Multiple Indicator Cluster Survey, 2018/19",NaN,Original chained constant price data are resca...,2012.0,Country uses the 1993 System of National Accou...,Value added at basic prices (VAB),NaN,"2017, 2011",BPM6,Actual,Special trade system,Consolidated central government,Enhanced General Data Dissemination System (e-...,"Integrated household survey (IHS), 2011",NaN,NaN,2010.0,2017.0,DZ,DZ,Algeria,Algeria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,PSE,West Bank and Gaza,Lower middle income,Middle East & North Africa,NaN,NaN,Israeli new shekel,2017,"Multiple Indicator Cluster Survey, 2019/20",NaN,2015,NaN,Country uses the 2008 System of National Accou...,NaN,NaN,Annual estimation for 2011-2017,BPM6,NaN,NaN,Consolidated central government,Special Data Dissemination Standard (SDDS),"Expenditure survey/budget survey (ES/BS), 2016/17",NaN,2010,2013.0,2018.0,PS,GZ,West Bank and Gaza,West Bank and Gaza
261,WLD,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World aggregate.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1W,1W,World,World
262,YEM,Republic of Yemen,Low income,Middle East & North Africa,IDA,NaN,Yemeni rial,2004,"Demographic and Health Survey, 2013",NaN,1990,NaN,Country uses the 1993 System of National Accou...,Value added at basic prices (VAB),1990–96,Annual estimation for 2011-2013,BPM5 (Converted into BPM6 by IMF),Estimate,Special trade system,NaN,Enhanced General Data Dissemination System (e-...,"Expenditure survey/budget survey (ES/BS), 2014",NaN,NaN,2012.0,2015.0,YE,RY,"Yemen, Rep.",Yemen
263,ZMB,Republic of Zambia,Lower middle income,Sub-Saharan Africa,IDA,HI

Establishing Connection

In [ ]:
conn = psycopg2.connect(**conString)
cur = conn.cursor()

In [ ]:
cur.execute("""SELECT * FROM edx.wrld_bnk_indc""")
res = cur.fetchall()

df = pd.DataFrame(res)

In [ ]:
df.columns=['country_name','country_code','year','edu_pcnt_scnd_bud','sch_enr_sec','edu_pcnt_prim_bud','edu_pcnt_gdp_pstudcapita_scnd','edu_pcnt_gdp_pstudcapita_prim',
             'edu_pcnt_ter_bud','edu_pcnt_bud','sch_enr_pri','edu_pcnt_gdp_pstud_capita_ter','edu_pcnt_gdp']

df

In [ ]:
msno.matrix(df)

In [ ]:
import klib

Closing Connection

In [ ]:
conn.close()